# lesson1: Convolutional Neural Networks with dogscats

Let's classify images using deep learning and submit the result to Kaggle!

## Prerequisite

In [5]:
# make some Python3 functions available on Python2
from __future__ import division, print_function

import sys
print(sys.version_info)

import theano
print(theano.__version__)

import keras
print(keras.__version__)

sys.version_info(major=3, minor=5, micro=3, releaselevel='final', serial=0)
0.9.0
2.0.4


This notebook assumes Keras with Theano backend.
- TODO: make TensorFlow version as another notebook

It also assumes that you will run it on either one of these two cases:
- Floydhub (--env theano:py2 -> Theano rel-0.8.2 + Keras 1.2.2 on Python2)
- local conda virtual environment (Theano 0.9.0 + Keras 2.0.4 on Python3)

Refer to [this FloydHub document](http://docs.floydhub.com/home/environments/) for available FloydHub environments.

## Setup

Make sure to have these files in the parent directory of the directory where you execute this notebook.

- available [in the official repo](https://github.com/fastai/courses/tree/master/deeplearning1/nbs) for Keras1 on Python2 (rename from original files)
    - utils_keras1.py
    - vgg16_keras1.py
    - vgg16bn_keras1.py
- available [in the unofficial repo](https://github.com/roebius/deeplearning1_keras2/tree/master/nbs) for Keras2 on Python3
    - utils.py
    - vgg16.py
    - vgg16bn.py

The directory structure looks like this. Please modifiy the symlinks according to your environment.

- (*) only for FloydHub
- (**) only for local

```
floyd_requirements.txt (*)
floydhub.data.unzip/   (*)
    (TODO)
floydhub.data.zipped/  (*)
    (TODO)
lesson1/
    data/ (**)
        redux/
            train/
                cat.437.jpg
                dog.9924.jpg
                ...
            test/
                231.jpg
                325.jpg
                ...
    dogscats_run.ipynb
    floyd_requirements.txt -> ../floyd_requirements.txt (*)
    utils.py -> ../utils(_keras1).py
    vgg16.py -> ../vgg16(_keras1).py
    vgg16bn.py -> ../vgg16bn(_keras1).py
utils.py
utils_keras1.py
vgg16.py
vgg16_keras1.py
vgg16bn.py
vgg16bn_keras1.py
```

In [6]:
# check current directory
%pwd
%ls

# see some files are loaded fine
%cat floyd_requirements.txt

# check no Keras2 specific function is used (when Keras1 is used)
%cat utils.py

__pycache__/            submission1.csv         vgg16bn.py@
dogscats_run.ipynb      utils.py@
floyd_requirements.txt@ vgg16.py@
bcolz
from __future__ import division,print_function
import math, os, json, sys, re

# import cPickle as pickle  # Python 2
import pickle  # Python3

from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from operator import itemgetter, attrgetter, methodcaller
from collections import OrderedDict
import itertools
from itertools import chain

import pandas as pd
import PIL
from PIL import Image
from numpy.random import random, permutation, randn, normal, uniform, choice
from numpy import newaxis
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
import bcolz
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

from IPython.lib.display import FileLink

import theano
from theano import shared, t

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()

LESSON_HOME_DIR = current_dir

# FloydHub
DATA_HOME_DIR = "/input/dogscats/"
OUTPUT_HOME_DIR = "/output/"

# local
#DATA_HOME_DIR = current_dir+'/data/redux'

In [8]:
#import modules
from utils import *
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

## Prepare data

(omit) prepared in floydhub.

## FloydHub

The cell below shows how to update data to FloydHub.

```
cd ../floydhub.data.zipped/; pwd

# empty
ls -l

wget http://files.fast.ai/files/dogscats.zip

# upload the zipped dataset to floydnet, and create a floydnet dataset
floyd data init dogscats.zipped
floyd data upload
```

In [1]:
# FloydHub: check data
%ls /input/dogscats/

ls: /input/dogscats/: No such file or directory


## local

https://github.com/kazuhirokomoda/deep_learning/blob/master/fast.ai/17flowers_data.ipynb

## Finetuning and Training

In [11]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test1/' #We use all the test data

# in FloydHub, data needs to be output under /output
# if results_path cannot be created, execute mkdir directly in the terminal
results_path = OUTPUT_HOME_DIR + '/results/'
%mkdir results_path

train_path = path + '/train/'
valid_path = path + '/valid/'

[Errno 2] No such file or directory: '/input/dogscats/'
/Users/kkomoda/git/deep_learning/fast.ai/lesson1


## Use a pretrained VGG model with our Vgg16 class

In [12]:
# As large as you can, but no larger than 64 is recommended.
#batch_size = 8
batch_size = 64

no_of_epochs=3

The original pre-trained Vgg16 class classifies images into one of the 1000 categories. This number of categories depends on the dataset which Vgg16 was trained with. (http://image-net.org/challenges/LSVRC/2014/browse-synsets)

In order to classify images into the categories which we prepare (2 categories of dogs/cats, in this notebook), *fine-tuning* technology is useful. It:
- keeps the most weights from the pre-trained Vgg16 model, but modifies only a few parts of the weights
- changes the dimension of the output layer (from 1000 to 2, in this notebook) 

In [10]:
vgg = Vgg16()

In [ ]:
# Grab a few images at a time for training and validation.
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)

# Finetune: note that the vgg model is compiled inside the finetune method.
vgg.finetune(batches)

In [7]:
# Fit: note that we are passing in the validation dataset to the fit() method
# For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    vgg.fit(batches, val_batches, batch_size, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print("Completed %s fit operations" % no_of_epochs)

Found 1020 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Epoch 1/1
128/128 [==============================] - 670s - loss: 2.0221 - acc: 0.5420 - val_loss: 0.6882 - val_acc: 0.7765


## Generate Predictions

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
print(preds[:5])

filenames = batches.filenames
print(filenames[:5])

In [ ]:
# You can verify the column ordering by viewing some images
from PIL import Image
Image.open(test_path + filenames[2])

In [ ]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

## Validate Predictions

Calculate predictions on validation set, so we can find correct and incorrect examples:

In [ ]:
vgg.model.load_weights(results_path+latest_weights_filename)

In [ ]:
val_batches, probs = vgg.test(valid_path, batch_size = batch_size)

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

In [ ]:
#Round our predictions to 0/1 to generate labels
our_predictions = probs[:,0]
our_labels = np.round(1-our_predictions)

(TODO) look at data to improve model

confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)

In [ ]:
plot_confusion_matrix(cm, val_batches.class_indices)

## Submit Predictions to Kaggle!

In [ ]:
#Load our test predictions from file
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
#Grab the dog prediction column
isdog = preds[:,1]
print("Raw Predictions: " + str(isdog[:5]))
print("Mid Predictions: " + str(isdog[(isdog < .6) & (isdog > .4)]))
print("Edge Predictions: " + str(isdog[(isdog == 1) | (isdog == 0)]))

In [ ]:
# sneaky trick to round down our edge predictions
# Swap all ones with .95 and all zeros with .05
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
from IPython.display import FileLink
%cd $LESSON_HOME_DIR
FileLink('data/redux/'+submission_file_name)